# make_Newport_topo

Using tiles downloaded from 
https://www.ngdc.noaa.gov/thredds/catalog/tiles/nthmp/tiled_19as/catalog.html

R.J. LeVeque, 1/30/25

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
import os

sys.path.insert(0,'../common_code')
import new_topotools  # with merge tools

### Function to adjust to MHW, make plot and topofile:

In [ ]:
def adjust_dz_plot_save(topo, name, dz=-2.05):
    """
    Adjust topo by dz, save the topofile as <name>.asc
    and create a plot saved as <name>.png

    To adjust from NAVD88 to MHW, dz = -2.05 seems right for Newport region from VDATUM
       https://vdatum.noaa.gov/vdatumweb/vdatumweb?a=191901220200922#Point
    Set dz=0 to create files for original NAVD88.
    """

    Z_mhw = topo.Z + dz
    topo_mhw = topo.crop() # create a copy
    topo_mhw.set_xyZ(topo.x, topo.y, Z_mhw)  # reset Z array

    fig,ax = subplots()
    topo_mhw.plot(limits=[-50,50],axes=ax,
            cb_kwargs={'extend':'both', 'shrink':0.7})
    title(name)
    fname = '%s.png' % name
    savefig(fname, bbox_inches='tight')
    print('Created ',fname)

    fname = '%s.asc' % name
    topo_mhw.write(fname,topo_type=3,header_style='asc',Z_format='%12.5e')
    print('Created ',fname)

    return topo_mhw
    

In [ ]:
topodir = '/Users/rjl/topo/CUDEM'

## Tile covering coastal Newport:

In [ ]:
extent = [-124.15, -123.01, 44.55, 44.68]
fname = 'ncei19_n44x75_w124x25_navd88_2021.nc'
topo1 = topotools.read_netcdf(os.path.join(topodir,fname), extent=extent,
                             coarsen=1, verbose=True)

In [ ]:
print('Extent: \n', topo1.extent)
print('Offset in x = %.3f 1/9" cells' % ((topo1.x[0]+124)*9*3600))
print('Offset in y = %.3f 1/9" cells' % ((topo1.y[0]-44)*9*3600))
print('These should be integers if aligned so DEM point is at integer lat-long')

### crop around Yaquina Bay at full 1/9" resolution

In [ ]:
extent = [-124.09, -124.0, 44.6, 44.635]
topo1a = topo1.crop(filter_region=extent)
print('Extent: \n', topo1a.extent)
print('Offset in x = %.3f 1/9" cells' % ((topo1a.x[0]+124)*9*3600))
print('Offset in y = %.3f 1/9" cells' % ((topo1a.y[0]-44)*9*3600))
print('These should be integers if aligned so DEM point is at integer lat-long')

In [ ]:
topo1a_mhw = adjust_dz_plot_save(topo1a, 'YaquinaBay_19s_mhw', dz=-2.05)

### Coarsen by 3 to obtain 1/3" DEM

In [ ]:
topo = topo1
istart = 2
jstart = 0
print('Using istart = %i, Offset in x = %.3f 1/3" cells' % (istart, (topo.x[istart]+124)*3*3600))
print('Using jstart = %i, Offset in y = %.3f 1/3" cells' % (jstart, (topo.y[jstart]-44)*3*3600))
print('These should be integers if aligned so COARSENED-BY-3 DEM hits integer lat-long')
eps = 0.5/(9*3600.)  # half a cell width
filter_region = [topo.x[istart] - eps, topo.x[-1] + eps, topo.y[jstart] - eps, topo.y[-1] + eps]
print('Crop using\n   filter_region = ',filter_region)

In [ ]:
topo1c = topo1.crop(filter_region=filter_region, coarsen=3)
print('Extent: \n', topo1c.extent)
print('Offset in x = %.3f 1/3" cells' % ((topo1c.x[0]+124)*3*3600))
print('Offset in y = %.3f 1/3" cells' % ((topo1c.y[0]-44)*3*3600))
print('These should be integers if aligned so DEM point is at integer lat-long')

In [ ]:
if 0:
    # plot and save in original NAVD88
    adjust_dz_plot_save(topo1c, 'Newport_13s', dz=0.)

In [ ]:
topo1c_mhw = adjust_dz_plot_save(topo1c, 'Newport_13s_mhw', dz=-2.05)

## Tile covering East of Newport (up the Yaquina River)

In [ ]:
extent = [-124.01, -123.9, 44.55, 44.68]
fname = 'ncei19_n44x75_w124x00_navd88_2021.nc'
topo2 = topotools.read_netcdf(os.path.join(topodir,fname), extent=extent,
                             coarsen=1, verbose=True)

In [ ]:
print('Extent: \n', topo2.extent)
print('Offset in x = %.3f 1/9" cells' % ((topo2.x[0]+124)*9*3600))
print('Offset in y = %.3f 1/9" cells' % ((topo2.y[0]-44)*9*3600))
print('These should be integers if aligned so DEM point is at integer lat-long')

### Coarsen by 3 to obtain 1/3" DEM

In [ ]:
topo = topo2
istart = 0
jstart = 0
print('Using istart = %i, Offset in x = %.3f 1/3" cells' % (istart, (topo.x[istart]+124)*3*3600))
print('Using jstart = %i, Offset in y = %.3f 1/3" cells' % (jstart, (topo.y[jstart]-44)*3*3600))
print('These should be integers if aligned so COARSENED-BY-3 DEM hits integer lat-long')
eps = 0.5/(9*3600.)  # half a cell width
filter_region = [topo.x[istart] - eps, topo.x[-1] + eps, topo.y[jstart] - eps, topo.y[-1] + eps]
print('Crop using\n   filter_region = ',filter_region)

In [ ]:
topo2c = topo2.crop(filter_region=filter_region, coarsen=3)
print('Extent: \n', topo1c.extent)
print('Offset in x = %.3f 1/3" cells' % ((topo2c.x[0]+124)*3*3600))
print('Offset in y = %.3f 1/3" cells' % ((topo2c.y[0]-44)*3*3600))
print('These should be integers if aligned so DEM point is at integer lat-long')

In [ ]:
topo2c_mhw = adjust_dz_plot_save(topo2c, 'NewportE_13s_mhw', dz=-2.05)